# Hidden Markov Models (HMM's) for Sequence Tagging

## Generative Models
Let $\mathcal{V}$ be a vocabulary and $\mathcal{K}$ a set of tags, define 

$$
S = \{(x_1, \dots, x_n, y_1, \dots, y_n):x_i \in \mathcal{V}, y_i \in \mathcal{K}\}
$$

A generative model is a function $p$ satisfying

- $p(x_1, \dots, x_n, y_1, \dots, y_n)>0, \forall (x_1, \dots, x_n, y_1, \dots, y_n) \in S$
- $\sum \limits_{(x_1, \dots, x_n, y_1, \dots, y_n) \in S} p(x_1, \dots, x_n, y_1, \dots, y_n) = 1$

Within these conditions, we define the tagger $f:\mathcal{V}^n \rightarrow \mathcal{K}^n$ as

$$
f(x_1, x_2, \dots, x_n) = \arg\,\max\limits_{y_1, \dots, y_n} p(x_1, \dots, x_n, y_1, \dots, y_n)
$$

## Trigram HMM's

### Definition

A Trigram HMM's is a finite vocabulary $\mathcal{V}$, a finite set of tags $\mathcal{K}$ along with the following parameters:

- $q(u| v, s)$, where $u, v, s \in \mathcal{K}$. This can be thought as the probability of seeing the tag $u$ immediately after seeing the tags $v$ and $s$;
- $e(x| s)$, where $x \in \mathcal{V}$ and $s \in \mathcal{K}$. This can be thought as the probability of seeing the word $x$ paired with the tag $s$.

By taking the set $S$ as before we define

$$
p(x_1, \dots, x_n, y_1, \dots, y_n) = \prod_{i=1}^{n+1}q(y_i|y_{i-1},y_{i-2})\prod_{i=1}^n e(x_i|y_i)
$$

Note that, we are assuming our sentences are second order Markov sentences, i.e, the state at time $t$ depends only on the states at time $t-1$ and $t-2$. That's the reason why the probability function $p$ is reduced to the form above.

### Parameter Estimation

Define $c(u, v, s)$ to be the number of times the sequence of three states $(u,v,s)$ appears on the training set, similarly $c(u, v)$ is the number fo times the pair $(u,v)$ appears and finally, $c(u)$ counts the number of times the state $s$ shows up. Moreover, define $c(s \leadsto x)$ to be the number of times the word $x$ is paired with state $s$ and $c(x)$ the number of times we see the word $x$ on the training set. Then, we set

$$
q(u|v, s) = \frac{c(u, v, s)}{c(v, s)} \;\;\;\;\;\;\;\text{and} \;\;\;\;\;\;\; e(x|s) = \frac{c(s \leadsto x)}{c(x)}
$$

### Decoding

In order to tag the sequence we need to evaluate the function $f$, the problem is that if the sentece has length $n$ then there are $|\mathcal{K}|^n$ different ways to tag it. This means that testing every single one is not a wise choice, alternatively though, one can use the Viterbi algorithm. 

First define $r(y_{-1}, y_0, y_1, \dots, y_k)$ to be the truncated version of the function $p$ defined previously, where $y_{-1}=y_0=*$ a special symbol to denote invalid words. Moreover denote, $\mathcal{K}_k=\mathcal{K}, \forall k=1, \dots, n$ and $\mathcal{K}_{-1}=\mathcal{K}_0=\{*\}$. Thus, we can also set

$$
S(k, u, v) = \{\langle y_{-1}y_0y_1\dots y_k \rangle|y_{k-1}=u, y_{k}=v, y_i \in \mathcal{K}_i, \forall i \ne k-1, k\}
$$

In other words, $S$ is the set of all sequences of length $k$ that ends with the bigram $(u, v)$. Finally, also set

$$
\pi(k, u, v) = \max \limits_{\langle y_{-1}y_0y_1\dots y_k \rangle \in S(k, u, v)} r(y_{-1}, y_0, y_1, \dots, y_k)
$$

Now, we just need to realize that the following recursion hold true:
$$
\pi(k, u, v) = \max \limits_{w \in \mathcal{K}_{k-2}} \pi(k-1, w, v)\times q(v|w,u) \times e(x_k|v), \;\text{where}\; \pi(0, *, *) = 1
$$

And, after denoting $y_{n+1} = STOP$ another special symbol to represent the end of the sentence, the decoding can be done as:
$$
\max\limits_{y_1, y_2, \dots, y_{n+1}} p(x_1, x_2, \dots, x_n) = \max\limits_{u \in \mathcal{K}_{n-1}, v \in \mathcal{K}_n} \pi(n, u, v) \times q(STOP|u, v)
$$

If one adds a pointer to track the indexes of the sequence of states that maximizes the probability $p$, we can easily recover the label in $\mathcal{O}(n|\mathcal{K}|^3)$.

### Drawbacks

The generative models do not handle unseen words, then it is necessary to map low frequency words to pseudo-words.

## References

All the ideas presented here are discussed in details in this link: http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf

In [1]:
import nltk
import time
from pprint import pprint
from functools import reduce
from operator import iconcat
from nltk.tag.hmm import HiddenMarkovModelTrainer
from sklearn import metrics

## Dataset

We will use the MIT movie corpus as a toy dataset to perform named entity recognition (NER), the data is available here: https://groups.csail.mit.edu/sls/downloads/movie/

In [2]:
def process_file(fname:str)->list:
    with open(fname, "r") as f:
        data = f.read()
    data = data.split("\n\n")
    data = list(map(lambda x:x.split("\n"), data))
    data = list(map(lambda x:[tuple(s.split("\t"))[::-1] for s in x], data))
    return data

In [3]:
train = process_file("./datasets/engtrain.bio")

In [4]:
train[:3]

[[('what', 'O'),
  ('movies', 'O'),
  ('star', 'O'),
  ('bruce', 'B-ACTOR'),
  ('willis', 'I-ACTOR')],
 [('show', 'O'),
  ('me', 'O'),
  ('films', 'O'),
  ('with', 'O'),
  ('drew', 'B-ACTOR'),
  ('barrymore', 'I-ACTOR'),
  ('from', 'O'),
  ('the', 'O'),
  ('1980s', 'B-YEAR')],
 [('what', 'O'),
  ('movies', 'O'),
  ('starred', 'O'),
  ('both', 'O'),
  ('al', 'B-ACTOR'),
  ('pacino', 'I-ACTOR'),
  ('and', 'O'),
  ('robert', 'B-ACTOR'),
  ('deniro', 'I-ACTOR')]]

The format nltk uses to represent tagged sentences is quite simple, the training set is a list of sentences. Each sentence is a list of tuples, the first element is a word and the second is its corresponding tag. Also note that, the tag O represents that the corresponding word is not an entity, and B-SOMETHING represents the entity beginning and I-SOMETHING represents both the intermediate and last words of the entity.

In [5]:
train.pop() # Last sentence is empty

[('',)]

In [6]:
f"There are {len(train)} sentences in the training set"

'There are 9775 sentences in the training set'

In [7]:
def to_list(data:list)->list:
    return reduce(iconcat, data, [])

def split_words_n_tags(data:list)->tuple:
    words, tags = map(list, zip(*data))
    return words, tags

In [8]:
all_pairs = to_list(train)
all_words, all_tags = split_words_n_tags(all_pairs)

In [9]:
f"There are {len(set(all_words))} unique words in the training set"

'There are 6710 unique words in the training set'

In [10]:
f"There are {len(set(all_tags))} unique tags in the training set"

'There are 25 unique tags in the training set'

In [11]:
hist = nltk.probability.FreqDist(all_tags)
hist.most_common()

[('O', 61008),
 ('B-GENRE', 4354),
 ('I-TITLE', 3495),
 ('I-ACTOR', 3474),
 ('B-ACTOR', 3220),
 ('B-YEAR', 2858),
 ('I-YEAR', 2456),
 ('B-TITLE', 2376),
 ('B-RATING', 2007),
 ('B-PLOT', 1927),
 ('B-RATINGS_AVERAGE', 1869),
 ('I-DIRECTOR', 1850),
 ('B-DIRECTOR', 1720),
 ('I-PLOT', 1687),
 ('I-RATINGS_AVERAGE', 1673),
 ('I-RATING', 840),
 ('I-GENRE', 786),
 ('I-SONG', 446),
 ('B-CHARACTER', 385),
 ('I-CHARACTER', 342),
 ('B-SONG', 245),
 ('B-REVIEW', 221),
 ('I-REVIEW', 132),
 ('B-TRAILER', 113),
 ('I-TRAILER', 7)]

The class $\textit{nltk.probability.FreqDist}$ builds a histogram that tells us how many times each tag appeared on the training set. We can already expect that the tags with low frequency will be the ones the model will have more trouble during testing time.

## Training

It is quite easy to train an HMM using the nltk API, the lack of hyperparameters contribuites a lot to it. In our case, the method we are about to call is the $\textit{train_supervised}$, given the way we processed the sentences is enough to feed the variable $\textit{train}$ directly to the model.

In [12]:
tic = time.time()
hmm = HiddenMarkovModelTrainer().train_supervised(train)
toc = time.time()
f"HMM took {(toc-tic):.5f} seconds to train"

'HMM took 0.13901 seconds to train'

Let's pick a sentence in the test set to see what we can get from the trained model.

In [13]:
test = process_file("./datasets/engtest.bio")
test.pop()

[('',)]

In [14]:
toy = [w for w,t in test[2]]
toy

['list', 'the', 'five', 'star', 'rated', 'movies', 'starring', 'mel', 'gibson']

In [15]:
hmm.point_entropy(toy)

array([0.2529096 , 0.59349388, 0.49746641, 1.0173373 , 1.53786785,
       0.94886663, 0.5767745 , 0.57020132, 1.25574454])

In [16]:
hmm.entropy(toy)

4.048307571135144

The $\textit{point_entropy}$ method gives the amount of uncertainty the model is about the tags assign to each word in the sentence. We can also recover the entropy for the whole sentence using the $\textit{entropy}$ method.

Let's tag this sequence and compare it with the actual states

In [17]:
hmm.tag(toy)

[('list', 'O'),
 ('the', 'O'),
 ('five', 'B-RATINGS_AVERAGE'),
 ('star', 'I-RATINGS_AVERAGE'),
 ('rated', 'I-RATINGS_AVERAGE'),
 ('movies', 'O'),
 ('starring', 'O'),
 ('mel', 'B-ACTOR'),
 ('gibson', 'I-ACTOR')]

In [18]:
test[2]

[('list', 'O'),
 ('the', 'O'),
 ('five', 'B-RATINGS_AVERAGE'),
 ('star', 'I-RATINGS_AVERAGE'),
 ('rated', 'O'),
 ('movies', 'O'),
 ('starring', 'O'),
 ('mel', 'B-ACTOR'),
 ('gibson', 'I-ACTOR')]

We shall note that the word "rated" has the highest entropy, thus, is the one in the sentence the model is more uncertain about. It is also the only label the model made a mistake, but it is important to keep in mind that high confidence does really mean the prediction is right, as an example take the case down below:

In [19]:
toy = [w for w,t in test[0]]
toy

['are', 'there', 'any', 'good', 'romantic', 'comedies', 'out', 'right', 'now']

In [20]:
hmm.point_entropy(toy)

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [21]:
hmm.tag(toy)

[('are', 'O'),
 ('there', 'O'),
 ('any', 'O'),
 ('good', 'O'),
 ('romantic', 'B-GENRE'),
 ('comedies', 'I-GENRE'),
 ('out', 'O'),
 ('right', 'O'),
 ('now', 'O')]

In [22]:
test[0]

[('are', 'O'),
 ('there', 'O'),
 ('any', 'O'),
 ('good', 'O'),
 ('romantic', 'B-GENRE'),
 ('comedies', 'I-GENRE'),
 ('out', 'O'),
 ('right', 'B-YEAR'),
 ('now', 'I-YEAR')]

The model is high confident about all its predictions but it got the last two words wrongly, maybe due to the fact that the word "out" is often followed by the year the movie will be released.

## Testing

Let's test our model on all sentences in the testing set to see how it performed. Firstly, we will unlabel the testing set to simulate when real predictions are desired.

In [23]:
def retrive_sents(data:list)->list:
    return list(map(lambda x:[w for w,t in x], data))

In [24]:
_, labels = split_words_n_tags(to_list(test))
unlabeled_sents = retrive_sents(test)

In [25]:
unlabeled_sents[:3]

[['are',
  'there',
  'any',
  'good',
  'romantic',
  'comedies',
  'out',
  'right',
  'now'],
 ['show', 'me', 'a', 'movie', 'about', 'cars', 'that', 'talk'],
 ['list',
  'the',
  'five',
  'star',
  'rated',
  'movies',
  'starring',
  'mel',
  'gibson']]

By calling the method $\textit{tag_sents}$ we can tag all sentences at once using the HMM.

In [26]:
tic = time.time()
preds = hmm.tag_sents(unlabeled_sents)
toc = time.time()
f"HMM took {(toc-tic):.5f} seconds to tag all sequences in the testing set"

'HMM took 2.54075 seconds to tag all sequences in the testing set'

In [27]:
_, preds = split_words_n_tags(to_list(preds))
preds[:10]

['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'O', 'O', 'O']

In [28]:
labels[:10]

['O', 'O', 'O', 'O', 'B-GENRE', 'I-GENRE', 'O', 'B-YEAR', 'I-YEAR', 'O']

Now we have a list with all predicted tags regardless the sentence, since we are not interested in sentence-wise performance. The $\textit{labels}$ variable is a list paired with the $\textit{preds}$, so we can call the $\textit{classification_report}$ function from the $\textit{sklearn}$ API to measure the accuracy of the HMM's predictions.

In [29]:
pprint(metrics.classification_report(labels, preds))

/home/gabriel/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('                   precision    recall  f1-score   support\n'
 '\n'
 '          B-ACTOR       0.87      0.82      0.84       812\n'
 '      B-CHARACTER       0.40      0.37      0.38        90\n'
 '       B-DIRECTOR       0.80      0.59      0.68       456\n'
 '          B-GENRE       0.92      0.91      0.92      1117\n'
 '           B-PLOT       0.58      0.47      0.52       491\n'
 '         B-RATING       0.97      0.90      0.94       500\n'
 'B-RATINGS_AVERAGE       0.88      0.73      0.80       451\n'
 '         B-REVIEW       0.17      0.20      0.18        56\n'
 '           B-SONG       0.33      0.24      0.28        54\n'
 '          B-TITLE       0.64      0.45      0.53       562\n'
 '        B-TRAILER       0.84      0.87      0.85        30\n'
 '           B-YEAR       0.93      0.82      0.87       720\n'
 '          I-ACTOR       0.91      0.76      0.83       862\n'
 '      I-CHARACTER       0.45      0.32      0.38        75\n'
 '       I-DIRECTOR       0.86    

In the previous table we can see the precision, recall and f1-score for each tag. The B-CHARACTER, B-REVIEW, B-SONG, I-CHARACTER, I-REVIEW and I-TRAILER were the most difficult tags to model by the HMM, oddly enough, they are also the ones that have the smallest support. Further analysis is necessary to tell if this model is indeed robust to make actual predictions for this dataset, but since this is an introductory example we will stop here :)